# Running Native Functions


Two of the previous notebooks showed how to [execute semantic functions inline](./03-semantic-function-inline.ipynb) and how to [run prompts from a file](./02-running-prompts-from-file.ipynb).

In this notebook, we'll show how to use native functions from a file. We will also show how to call semantic functions from native functions.

This can be useful in a few scenarios:

- Writing logic around how to run a prompt that changes the prompt's outcome.
- Using external data sources to gather data to concatenate into your prompt.
- Validating user input data prior to sending it to the LLM prompt.

Native functions are defined using standard Python code. The structure is simple, but not well documented at this point.

The following examples are intended to help guide new users towards successful native & semantic function use with the SK Python framework.


Prepare a semantic kernel instance first, loading also the AI service settings defined in the [Setup notebook](00-getting-started.ipynb):


In [1]:
!pip install semantic-kernel==0.9.2b1

In [4]:
from services import Service

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = Service.AzureOpenAI

In [5]:
import semantic_kernel as sk
import semantic_kernel.connectors.ai.open_ai as sk_oai
from semantic_kernel.prompt_template.input_variable import InputVariable

kernel = sk.Kernel()

if selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    service_id = "aoai_chat"  # used later in the notebook
    azure_chat_service = AzureChatCompletion(
        service_id=service_id, deployment_name=deployment, endpoint=endpoint, api_key=api_key
    )  # set the deployment name to the value of your chat model
    kernel.add_service(azure_chat_service)

# Configure OpenAI service
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    api_key, org_id = sk.openai_settings_from_dot_env()
    service_id = "oai_chat"  # used later in the notebook
    oai_chat_service = OpenAIChatCompletion(
        service_id=service_id, ai_model_id="gpt-4-turbo-1106", api_key=api_key, org_id=org_id
    )
    kernel.add_service(oai_chat_service)

Let's create a **native** function that gives us a random number between 3 and a user input as the upper limit. We'll use this number to create 3-x paragraphs of text when passed to a semantic function.


First, let's create our native function.


In [6]:
import random
from semantic_kernel.functions import kernel_function


class GenerateNumberPlugin:
    """
    Description: Generate a number between 3-x.
    """

    @kernel_function(
        description="Generate a random number between 3-x",
        name="GenerateNumberThreeOrHigher",
    )
    def generate_number_three_or_higher(self, input: str) -> str:
        """
        Generate a number between 3-<input>
        Example:
            "8" => rand(3,8)
        Args:
            input -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            return str(random.randint(3, int(input)))
        except ValueError as e:
            print(f"Invalid input {input}")
            raise e

Next, let's create a semantic function that accepts a number as `{{$input}}` and generates that number of paragraphs about two Corgis on an adventure. `$input` is a default variable semantic functions can use.


In [7]:
prompt = """
Write a short story about two Corgis on an adventure.
The story must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$input}} paragraphs long. It must be this length.
"""

if selectedService == Service.OpenAI:
    execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo-1106",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id=deployment,
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = sk.PromptTemplateConfig(
    template=prompt,
    name="story",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True),
    ],
    execution_settings=execution_settings,
)

corgi_story = kernel.create_function_from_prompt(
    function_name="CorgiStory",
    plugin_name="CorgiPlugin",
    prompt_template_config=prompt_template_config,
)

generate_number_plugin = kernel.import_plugin_from_object(GenerateNumberPlugin(), "GenerateNumberPlugin")

In [8]:
# Run the number generator
generate_number_three_or_higher = generate_number_plugin["GenerateNumberThreeOrHigher"]
number_result = await generate_number_three_or_higher(kernel, input=6)
print(number_result)

5


In [9]:
story = await corgi_story.invoke(kernel, input=number_result.value)

_Note: depending on which model you're using, it may not respond with the proper number of paragraphs._

In [10]:
print(f"Generating a corgi story exactly {number_result.value} paragraphs long.")
print("=====================================================")
print(story)

Generating a corgi story exactly 5 paragraphs long.
Once upon a time in a small town called Pawsomeville, there lived two adventurous Corgis named Charlie and Daisy. They were the best of friends, always seeking new and exciting experiences. One sunny morning, they decided to embark on a thrilling adventure to explore the nearby enchanted forest.

As they entered the forest, Charlie and Daisy were greeted by the sweet melody of chirping birds and the rustling of leaves beneath their paws. They marveled at the tall trees and colorful flowers that surrounded them. Excitement filled their hearts as they ventured deeper into the unknown.

In the heart of the forest, they stumbled upon a sparkling river. Its crystal-clear water danced under the golden sunlight, inviting them to take a refreshing dip. Charlie and Daisy eagerly jumped in, splashing and playing together. They were reminded of the importance of having fun and cherishing each moment.

After their playful swim, they stumbled upon

## Kernel Functions with Annotated Parameters

That works! But let's expand on our example to make it more generic.

For the native function, we'll introduce the lower limit variable. This means that a user will input two numbers and the number generator function will pick a number between the first and second input.

We'll make use of the Python's `Annotated` class to hold these variables.


In [11]:
import sys
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    OpenAIChatCompletion,
)

if sys.version_info >= (3, 9):
    from typing import Annotated
else:
    from typing_extensions import Annotated

kernel = sk.Kernel()

if selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    deployment, api_key, endpoint = sk.azure_openai_settings_from_dot_env()
    service_id = "aoai_chat"  # used later in the notebook
    azure_chat_service = AzureChatCompletion(
        service_id=service_id, deployment_name=deployment, endpoint=endpoint, api_key=api_key
    )  # set the deployment name to the value of your chat model
    kernel.add_service(azure_chat_service)

# Configure OpenAI service
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    api_key, org_id = sk.openai_settings_from_dot_env()
    service_id = "oai_chat"  # used later in the notebook
    oai_chat_service = OpenAIChatCompletion(
        service_id=service_id, ai_model_id="gpt-4-turbo-1106", api_key=api_key, org_id=org_id
    )
    kernel.add_service(oai_chat_service)

Let's start with the native function. Notice that we're add the `@kernel_function` decorator that holds the name of the function as well as an optional description. The input parameters are configured as part of the function's signature, and we use the `Annotated` type to specify the required input arguments.


In [12]:
import random
from semantic_kernel.functions import kernel_function


class GenerateNumberPlugin:
    """
    Description: Generate a number between a min and a max.
    """

    @kernel_function(
        name="GenerateNumber",
        description="Generate a random number between min and max",
    )
    def generate_number(
        self,
        min: Annotated[int, "the minimum number of paragraphs"],
        max: Annotated[int, "the maximum number of paragraphs"] = 10,
    ) -> Annotated[int, "the output is a number"]:
        """
        Generate a number between min-max
        Example:
            min="4" max="10" => rand(4,8)
        Args:
            min -- The lower limit for the random number generation
            max -- The upper limit for the random number generation
        Returns:
            int value
        """
        try:
            return str(random.randint(min, max))
        except ValueError as e:
            print(f"Invalid input {min} and {max}")
            raise e

In [13]:
generate_number_plugin = kernel.import_plugin_from_object(GenerateNumberPlugin(), "GenerateNumberPlugin")
generate_number = generate_number_plugin["GenerateNumber"]

Now let's also allow the semantic function to take in additional arguments. In this case, we're going to allow the our CorgiStory function to be written in a specified language. We'll need to provide a `paragraph_count` and a `language`.


In [14]:
prompt = """
Write a short story about two Corgis on an adventure.
The story must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$paragraph_count}} paragraphs long
- Be written in this language: {{$language}}
"""

if selectedService == Service.OpenAI:
    execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo-1106",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id=deployment,
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = sk.PromptTemplateConfig(
    template=prompt,
    name="summarize",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="paragraph_count", description="The number of paragraphs", is_required=True),
        InputVariable(name="language", description="The language of the story", is_required=True),
    ],
    execution_settings=execution_settings,
)

corgi_story = kernel.create_function_from_prompt(
    function_name="CorgiStory",
    plugin_name="CorgiPlugin",
    prompt_template_config=prompt_template_config,
)

Let's generate a paragraph count.


In [15]:
result = await generate_number.invoke(kernel, min=1, max=5)
num_paragraphs = result.value
print(f"Generating a corgi story {num_paragraphs} paragraphs long.")

Generating a corgi story 1 paragraphs long.


We can now invoke our corgi_story function using the `kernel` and the keyword arguments `paragraph_count` and `language`.

In [16]:
# Pass the output to the semantic story function
desired_language = "Spanish"
story = await corgi_story.invoke(kernel, paragraph_count=num_paragraphs, language=desired_language)

In [17]:
print(f"Generating a corgi story {num_paragraphs} paragraphs long in {desired_language}.")
print("=====================================================")
print(story)

Generating a corgi story 1 paragraphs long in Spanish.
Había una vez, en un pequeño pueblo, dos adorables corgis llamados Coco y Max. Eran los mejores amigos y siempre estaban juntos, pero un día sintieron la necesidad de aventurarse más allá de su hogar. Decidieron explorar el hermoso bosque que se extendía más allá de las colinas verdes.

Excitados, Coco y Max se adentraron en el bosque, saltando y corriendo entre los árboles. Descubrieron hermosas flores y mariposas de colores brillantes que revoloteaban a su alrededor. Pero mientras continuaban su viaje, se encontraron con un pequeño riachuelo que les bloqueaba el camino.

Sin embargo, en lugar de rendirse, Coco y Max trabajaron juntos para encontrar una solución. Max saltó de roca en roca, mientras Coco lo esperaba pacientemente al otro lado. Finalmente, llegaron al otro lado, pero se encontraron con un acantilado muy alto.

Coco y Max no se rindieron y buscaron una manera de superar el obstáculo. Coco tuvo una brillante idea y en

## Calling Native Functions within a Semantic Function

One neat thing about the Semantic Kernel is that you can also call native functions from within Prompt Functions!

We will make our CorgiStory semantic function call a native function `GenerateNames` which will return names for our Corgi characters.

We do this using the syntax `{{plugin_name.function_name}}`. You can read more about our prompte templating syntax [here](../../../docs/PROMPT_TEMPLATE_LANGUAGE.md).


In [18]:
import random
from semantic_kernel.functions import kernel_function


class GenerateNamesPlugin:
    """
    Description: Generate character names.
    """

    # The default function name will be the name of the function itself, however you can override this
    # by setting the name=<name override> in the @kernel_function decorator. In this case, we're using
    # the same name as the function name for simplicity.
    @kernel_function(description="Generate character names", name="generate_names")
    def generate_names(self) -> str:
        """
        Generate two names.
        Returns:
            str
        """
        names = {"Hoagie", "Hamilton", "Bacon", "Pizza", "Boots", "Shorts", "Tuna"}
        first_name = random.choice(list(names))
        names.remove(first_name)
        second_name = random.choice(list(names))
        return f"{first_name}, {second_name}"

In [19]:
generate_names_plugin = kernel.import_plugin_from_object(GenerateNamesPlugin(), plugin_name="GenerateNames")
generate_names = generate_names_plugin["generate_names"]

In [20]:
prompt = """
Write a short story about two Corgis on an adventure.
The story must be:
- G rated
- Have a positive message
- No sexism, racism or other bias/bigotry
- Be exactly {{$paragraph_count}} paragraphs long
- Be written in this language: {{$language}}
- The two names of the corgis are {{GenerateNames.generate_names}}
"""

In [21]:
if selectedService == Service.OpenAI:
    execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo-1106",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id=deployment,
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = sk.PromptTemplateConfig(
    template=prompt,
    name="corgi-new",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="paragraph_count", description="The number of paragraphs", is_required=True),
        InputVariable(name="language", description="The language of the story", is_required=True),
    ],
    execution_settings=execution_settings,
)

corgi_story = kernel.create_function_from_prompt(
    function_name="CorgiStoryUpdated",
    plugin_name="CorgiPluginUpdated",
    prompt_template_config=prompt_template_config,
)

In [22]:
result = await generate_number.invoke(kernel, min=1, max=5)
num_paragraphs = result.value

In [23]:
desired_language = "French"
story = await corgi_story.invoke(kernel, paragraph_count=num_paragraphs, language=desired_language)

In [24]:
print(f"Generating a corgi story {num_paragraphs} paragraphs long in {desired_language}.")
print("=====================================================")
print(story)

Generating a corgi story 5 paragraphs long in French.
Il était une fois, dans un petit village pittoresque situé au cœur de la campagne, deux adorables corgis nommés Boots et Shorts. Ces deux amis inséparables étaient connus pour leur énergie débordante et leur curiosité sans limites. Un jour, ils décidèrent de partir à l'aventure, à la recherche de nouvelles découvertes et de rencontres excitantes.

Les deux corgis marchaient joyeusement le long d'un sentier bordé de fleurs colorées. Ils se sentaient libres et enthousiastes à l'idée de ce voyage improvisé. Leurs petites pattes agiles les emmenèrent vers une forêt mystérieuse, où une légende racontait l'existence d'une cascade magique.

Boots et Shorts s'aventurèrent plus profondément dans la forêt, guidés par le bruit apaisant de l'eau qui tombait. Soudain, ils aperçurent une magnifique cascade scintillante, illuminée par les rayons du soleil. Les corgis étaient émerveillés par cette vue merveilleuse et se précipitèrent vers la cascad

### Recap

A quick review of what we've learned here:

- We've learned how to create native and prompt functions and register them to the kernel
- We've seen how we can use Kernel Arguments to pass in more custom variables into our prompt
- We've seen how we can call native functions within a prompt.
